<a href="https://colab.research.google.com/github/ryo1021/OCR/blob/main/theme_B_visual_structure/ocr_baseline/tesseract_ocr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pytesseract pillow
!apt-get install -y tesseract-ocr-jpn


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,902 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,468 kB]


In [3]:
from PIL import Image
import pytesseract

img = Image.open('スクリーンショット 2026-01-14 174845')
text = pytesseract.image_to_string(
    img,
    lang='jpn',
    config='--psm 6'
)
print(text)



FileNotFoundError: [Errno 2] No such file or directory: 'スクリーンショット 2026-01-14 174845'

In [ ]:
with open('ocr_output.txt', 'w') as f:
    f.write(text)


In [ ]:
import pytesseract
from PIL import Image
import json

img = Image.open('/content/スクリーンショット 2026-02-07 173412.png')

data = pytesseract.image_to_data(
    img,
    lang='jpn',
    config='--psm 6',
    output_type=pytesseract.Output.DICT
)


In [ ]:
elements = []

n = len(data['text'])
for i in range(n):
    txt = data['text'][i].strip()
    if txt == "":
        continue

    element = {
        "text": txt,
        "left": data['left'][i],
        "top": data['top'][i],
        "width": data['width'][i],
        "height": data['height'][i],
        "block": data['block_num'][i],
        "line": data['line_num'][i]
    }
    elements.append(element)

len(elements)

output = {
    "image_name": "sample_answer_01",
    "elements": elements
}

with open('ocr_elements.json', 'w', encoding='utf-8') as f:
    json.dump(output, f, ensure_ascii=False, indent=2)


In [ ]:
elements[:10]


[{'text': '@.P(s',
  'left': 19,
  'top': 5,
  'width': 184,
  'height': 91,
  'block': 1,
  'line': 1},
 {'text': 'て',
  'left': 216,
  'top': 24,
  'width': 63,
  'height': 43,
  'block': 1,
  'line': 1},
 {'text': ')',
  'left': 254,
  'top': 1,
  'width': 50,
  'height': 99,
  'block': 1,
  'line': 1},
 {'text': '、',
  'left': 309,
  'top': 32,
  'width': 61,
  'height': 51,
  'block': 1,
  'line': 1},
 {'text': 'ら',
  'left': 336,
  'top': 1,
  'width': 84,
  'height': 99,
  'block': 1,
  'line': 1},
 {'text': '(た',
  'left': 408,
  'top': 31,
  'width': 94,
  'height': 58,
  'block': 1,
  'line': 1},
 {'text': '、',
  'left': 477,
  'top': 1,
  'width': 50,
  'height': 99,
  'block': 1,
  'line': 1},
 {'text': 'し',
  'left': 523,
  'top': 37,
  'width': 70,
  'height': 45,
  'block': 1,
  'line': 1},
 {'text': '2',
  'left': 619,
  'top': 44,
  'width': 15,
  'height': 24,
  'block': 1,
  'line': 1},
 {'text': 'と',
  'left': 600,
  'top': 1,
  'width': 67,
  'height': 99,
  'block

In [ ]:
import cv2
import pytesseract
from PIL import Image
import json
import numpy as np

# === 画像読み込み（パスは適宜変更）===
img_path = "/content/スクリーンショット 2026-02-07 173412.png"
img = cv2.imread(img_path)

# === ① グレースケール化 ===
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# === ② コントラスト強調（手書き対策）===
gray = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX)

# === ③ ノイズ除去（薄い紙ノイズ対策）===
gray = cv2.GaussianBlur(gray, (5, 5), 0)

# === ④ 二値化（Otsu）===
_, bin_img = cv2.threshold(
    gray, 0, 255,
    cv2.THRESH_BINARY + cv2.THRESH_OTSU
)

# === ⑤ 反転（Tesseractは黒文字・白背景が得意）===
bin_img = 255 - bin_img

# PIL形式へ変換
pil_img = Image.fromarray(bin_img)

# === OCR 実行 ===
data = pytesseract.image_to_data(
    pil_img,
    lang="jpn",
    config="--psm 6",
    output_type=pytesseract.Output.DICT
)

# === 構造化 ===
elements = []
n = len(data["text"])

for i in range(n):
    txt = data["text"][i].strip()
    if txt == "":
        continue
    elements.append({
        "text": txt,
        "left": data["left"][i],
        "top": data["top"][i],
        "width": data["width"][i],
        "height": data["height"][i],
        "block": data["block_num"][i],
        "line": data["line_num"][i]
    })

# === 結果確認 ===
print("検出文字数:", len(elements))
elements[:20]


検出文字数: 142


[{'text': 'に',
  'left': 19,
  'top': 5,
  'width': 223,
  'height': 91,
  'block': 1,
  'line': 1},
 {'text': 'の',
  'left': 259,
  'top': 24,
  'width': 20,
  'height': 43,
  'block': 1,
  'line': 1},
 {'text': '2',
  'left': 309,
  'top': 31,
  'width': 286,
  'height': 57,
  'block': 1,
  'line': 1},
 {'text': 'ゆま',
  'left': 541,
  'top': 21,
  'width': 102,
  'height': 71,
  'block': 1,
  'line': 1},
 {'text': 'の',
  'left': 663,
  'top': 42,
  'width': 40,
  'height': 31,
  'block': 1,
  'line': 1},
 {'text': 'に',
  'left': 716,
  'top': 54,
  'width': 20,
  'height': 21,
  'block': 1,
  'line': 1},
 {'text': 'っ',
  'left': 79,
  'top': 83,
  'width': 28,
  'height': 34,
  'block': 1,
  'line': 1},
 {'text': '-',
  'left': 134,
  'top': 103,
  'width': 20,
  'height': 6,
  'block': 1,
  'line': 1},
 {'text': 'そ',
  'left': 159,
  'top': 91,
  'width': 29,
  'height': 38,
  'block': 1,
  'line': 1},
 {'text': '5',
  'left': 191,
  'top': 95,
  'width': 46,
  'height': 41,
  'bloc

In [ ]:
#ホワイトリストつき_vol1
from PIL import Image, ImageDraw, ImageFont
import pytesseract
import cv2
import numpy as np

# === 1. 画像読み込み ===
img_path = "/content/スクリーンショット 2026-02-07 173412.png"
orig = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# === 2. 前処理（重要）===
# コントラスト強調 + ノイズ除去
orig = cv2.adaptiveThreshold(
    orig, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY, 31, 5
)

# PILへ変換
pil_img = Image.fromarray(orig)

# === 3. OCR ===
data = pytesseract.image_to_data(
    pil_img,
    lang="jpn+eng",
    config=(
        "--psm 6 "
        "-c tessedit_char_whitelist="
        "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
        "+-=()^./xytsXYTS"
    ),
    output_type=pytesseract.Output.DICT
)

# === 4. bbox + text 描画 ===
draw = ImageDraw.Draw(pil_img)

n = len(data["text"])
elements = []

for i in range(n):
    txt = data["text"][i].strip()
    if txt == "":
        continue

    x = data["left"][i]
    y = data["top"][i]
    w = data["width"][i]
    h = data["height"][i]

    # bbox
    draw.rectangle(
        [(x, y), (x + w, y + h)],
        outline="red",
        width=2
    )

    # text（bboxの上）
    draw.text((x, y - 10), txt, fill="blue")

    elements.append({
        "text": txt,
        "left": x,
        "top": y,
        "width": w,
        "height": h,
        "block": data["block_num"][i],
        "line": data["line_num"][i]
    })

# === 5. 保存 & 表示 ===
out_path = "/content/ocr_bbox_overlay.png"
pil_img.save(out_path)

print(f"検出文字数: {len(elements)}")
out_path


検出文字数: 30


'/content/ocr_bbox_overlay.png'

In [ ]:
# ホワイトリストつき_vol1_改

from PIL import Image, ImageDraw
import pytesseract
import cv2
import json

# === 1. 画像読み込み ===
img_path = "/content/スクリーンショット 2026-02-07 173412.png"
orig = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

# === 2. 前処理 ===
orig = cv2.adaptiveThreshold(
    orig, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY, 41, 7   # ← 少しだけ緩める
)

pil_img = Image.fromarray(orig).convert("RGB")
draw = ImageDraw.Draw(pil_img)

# === 3. OCR ===
data = pytesseract.image_to_data(
    pil_img,
    lang="eng",  # whitelist前提なのでengでOK
    config=(
        "--oem 3 --psm 6 "
        "-c tessedit_char_whitelist="
        "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
        "+-=()^./xytsXYTS"
    ),
    output_type=pytesseract.Output.DICT
)

elements = []
n = len(data["text"])

# === 4. bbox + text 描画 ===
for i in range(n):
    txt = data["text"][i].strip()
    if txt == "":
        continue

    x, y, w, h = (
        data["left"][i],
        data["top"][i],
        data["width"][i],
        data["height"][i]
    )

    # 色分け
    if any(c.isdigit() for c in txt):
        color = "red"        # 数値・式
    elif any(c in "+-=^()" for c in txt):
        color = "orange"    # 演算子
    else:
        color = "blue"      # 変数

    draw.rectangle([(x, y), (x + w, y + h)], outline=color, width=2)
    draw.text((x, y - 12), txt, fill=color)

    elements.append({
        "text": txt,
        "bbox": [x, y, w, h],
        "block": data["block_num"][i],
        "line": data["line_num"][i]
    })

# === 5. JSON保存 ===
output = {
    "image": img_path,
    "num_elements": len(elements),
    "elements": elements
}

with open("ocr_elements.json", "w", encoding="utf-8") as f:
    json.dump(output, f, ensure_ascii=False, indent=2)

out_path = "/content/ocr_bbox_overlay.png"
pil_img.save(out_path)

print("検出要素数:", len(elements))
out_path


検出要素数: 24


'/content/ocr_bbox_overlay.png'